In [1]:
%load_ext autoreload
%autoreload 2
%cd ~/Research/Sriram/DeepSetRNN

/home/minh/Research/Sriram/DeepSetRNN


In [2]:
from collections import defaultdict, namedtuple
import os
import string
import logging
import importlib

import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from src import experiment_utils, utils

In [3]:
logging.info("starting logger")
_LOGGER = logging.getLogger('VisitTimeSeries')
_LOGGER.setLevel(logging.DEBUG)

# Reading data

In [4]:
_ROOT_DATA_PATH = 'data/MIMIC3database/'
_PROCESSED_DATA_PATH = os.path.join(_ROOT_DATA_PATH, 'processed/MIMIC3EachPerson')

In [19]:
train_series_df = pd.read_csv('train_timeseries_sorted_len.csv', index_col=['SUBJECT_ID', 'ADMISSION_NUM'])
train_labels_df = pd.read_csv('train_labels_sorted_len.csv', index_col=['SUBJECT_ID', 'ADMISSION_NUM'])

test_series_df = pd.read_csv('test_timeseries_sorted_len.csv', index_col=['SUBJECT_ID', 'ADMISSION_NUM'])
test_labels_df = pd.read_csv('test_labels_sorted_len.csv', index_col=['SUBJECT_ID', 'ADMISSION_NUM'])

/home/minh/.conda/envs/deepset/lib/python3.6/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [46]:
test_series_df.loc[test_indices_sort]

Hours  Capillary refill rate  \
SUBJECT_ID ADMISSION_NUM                                     
1001       1             -56.795833                    0.0   
           1             -55.545833                    0.0   
           1             -42.712500                    0.0   
           1             -33.962500                    0.0   
           1             -18.529167                    0.0   
           1              -0.795833                    0.0   
           1               1.204167                    0.0   
           1               2.204167                    0.0   
           1               2.287500                    0.0   
           1               3.204167                    0.0   
           1               4.204167                    0.0   
           1               5.204167                    0.0   
           1               6.204167                    0.0   
           1               7.204167                    0.0   
           1               8.204167                    0.0   
           1               9.204167                    0.0   
           1              10.204167                    0.0   
           1              11.204167                    0.0   
           1              12.204167                    0.0   
           1              12.370833                    0.0   
           1              12.454167                    0.0   
           1              12.537500                    0.0   
           1              12.620833                    0.0   
           1              12.704167                    0.0   
           1              12.787500                    0.0   
           1              12.870833                    0.0   
           1              12.954167                    0.0   
           1              13.037500                    0.0   
           1              13.120833                    0.0   
           1              13.204167                    0.0   
...                             ...                    ...   
995        2               9.728333                    0.0   
           2               9.978333                    0.0   
           2              10.478333                    0.0   
           2              10.561667                    0.0   
           2              10.628333                    0.0   
           2              11.478333                    0.0   
           2              11.728333                    0.0   
           2              12.478333                    0.0   
           2              12.728333                    0.0   
           2              13.478333                    0.0   
           2              14.228333                    0.0   
           2              14.411667                    0.0   
           2              14.478333                    0.0   
           2              15.445000                    0.0   
           2              15.478333                    0.0   
           2              15.728333                    0.0   
           2              16.478333                    0.0   
           2              17.478333                    0.0   
           2              17.961667                    0.0   
           2              18.011667                    0.0   
           2              18.478333                    0.0   
           2              19.478333                    0.0   
           2              20.478333                    0.0   
           2              20.978333                    0.0   
           2              21.478333                    0.0   
           2              22.478333                    0.0   
           2              23.478333                    0.0   
           2              41.478333                    0.0   
           2              65.811667                    0.0   
           2              86.978333                    0.0   

                          Diastolic blood pressure  Systolic blood pressure  \
SUBJECT_ID ADMISSION_NUM                    

In [32]:
indexer = pd.IndexSlice

train_indices_sort = np.load('{}/train_indices_sorted_by_len.npy'.format(_PROCESSED_DATA_PATH))
test_indices_sort = np.load('{}/test_indices_sorted_by_len.npy'.format(_PROCESSED_DATA_PATH))

In [40]:
train_indices = list(set(train_series_df.index).intersection(set(train_labels_df.index)))
test_indices = list(set(test_series_df.index).intersection(set(test_labels_df.index)))

In [36]:
train_indices = train_indices[train_indices_sort]
test_indices = test_indices[test_indices_sort]

IndexError: index 33845 is out of bounds for axis 0 with size 22055

In [39]:
len(train_indices)

22055

In [44]:
test_series_df.loc(indexer[train_indices[:2]])

TypeError: unhashable type: 'list'

In [31]:
train_indices

array([19808, 33845, 27626, ..., 11942, 26097, 13341])

In [8]:
train_indices = list(set(train_series_df.index).intersection(set(train_labels_df.index)))
test_indices = list(set(test_series_df.index).intersection(set(test_labels_df.index)))

In [17]:
idx = pd.IndexSlice


test_labels_df.loc[idx[test_indices[:2]]]

,,Diagnosis 4019,Diagnosis 4280,Diagnosis 41401,Diagnosis 42731,Diagnosis 25000,Diagnosis 5849,Diagnosis 2724,Diagnosis 51881,Diagnosis 53081,Diagnosis 5990,...,Diagnosis 570,Diagnosis 71590,Diagnosis 2869,Diagnosis 2763,Diagnosis 5770,Diagnosis V5865,Diagnosis 99662,Diagnosis 28860,Diagnosis 36201,Diagnosis 56210
SUBJECT_ID,ADMISSION_NUM,,,,,,,,,,,,,,,,,,,,,
11624,1,1,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32501,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_sorted_idx = np.argsort([ train_series_df.xs(idx, level=[0,1]).shape[0]
              for idx in train_indices])

test_sorted_idx = np.argsort([ test_series_df.xs(idx, level=[0,1]).shape[0]
              for idx in test_indices])

In [36]:
train_indices = np.array(train_indices)[train_sorted_idx]
test_indices = np.array(test_indices)[test_sorted_idx]


np.save('{}/train_indices_sorted.npy'.format(_PROCESSED_DATA_PATH), train_indices)
np.save('{}/test_indices_sorted.npy'.format(_PROCESSED_DATA_PATH), test_indices)

array([66, 65, 89, 75,  5, 48, 78, 31, 59,  1, 92, 44, 35, 73, 91, 41, 80,
       60, 30, 83, 54, 29, 16, 24, 10, 96, 86,  9, 27, 42, 53, 12, 46, 81,
       76,  7, 36, 39, 17, 49, 21, 93, 84, 38, 23, 71, 11, 20, 34, 97, 70,
       77, 25,  0,  8, 50, 51,  4, 74, 56, 18, 82, 57, 95, 26, 33, 87, 52,
       79, 88, 68, 72,  2, 28, 37,  6, 94, 14, 13, 19, 63, 47, 67, 69, 22,
       43, 58, 32, 61, 40, 15, 99,  3, 90, 45, 55, 85, 98, 62, 64])

# Training Model

In [13]:
_MODEL_LOG_ROOT_PATH = 'logs/MIMIC3/VisitTimeSeries'
_MODEL_ROOT_PATH = 'models/MIMIC3/VisitTimeSeries'

In [18]:
from src.model.lstm import LSTMClassifier

ModelArgs = namedtuple('HospitalVisitLSTMClassifier', 
                      ['hidden_dims',
                       'batch_size'
                       'n_epoch',
                       'lr',
                       'momentum',
                      'n_layers',
                      'dropout']
                      )
args = ModelArgs(
    hidden_dims=[500,200],
    batch_size=500,
    n_epoch = 5,
    lr = 0.1,
    n_layers=1,
    momentum = 0.9,
    dropout=0.5,
)

model_name = str(args)

model = LSTMClassifier(hidden_dims=args.hidden_dims,
                       input_dim=n_features,
                       n_class=n_class,
                       dropout=args.dropout)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
experiment_utils.setup_model_logger(_LOGGER, model_name, _MODEL_LOG_ROOT_PATH)

In [23]:
model = model.eval()
count = 0
total_loss = 0

for idx_batch in utils.generate_batch(test_indices[:10], 2):
    curr_series = 
    output = 
    
    curr_series = test_series_df.xs(idx_batch, level=[0,1])
    output = test_labels_df.xs(idx_batch, level=[0,1])
    
    logit = model(utils.to_tensor(curr_series))
    total_loss += loss_fn(logit, utils.to_tensor(output))
total_loss = total_loss.data.numpy()

_LOGGER.info("Initial Validation Loss: {}".format(total_loss/len(test_indices)))

KeyboardInterrupt: 

In [20]:
model = model.train()
_LOGGER.info("Training model...")
training_losses = []
for epoch in range(args.n_epoch):
    total_loss = 0
    for idx in train_indices[:100]:
        curr_series = train_series_df.xs(idx, level=[0,1])
        output = train_labels_df.xs(idx, level=[0,1])        
        logit = model(utils.to_tensor(curr_series))
        loss = loss_fn(logit, utils.to_tensor(output))
        total_loss += loss.data.numpy()
        loss.backward()
        optimizer.step()
    mean_loss = total_loss/len(train_indices[:100])
    _LOGGER.info("Epoch: {}, Loss: {}".format(epoch, mean_loss))
    training_losses.append(mean_loss)

INFO:VisitTimeSeries:Training model...
INFO:VisitTimeSeries:Epoch: 0, Loss: 636.6494873799384
INFO:VisitTimeSeries:Epoch: 1, Loss: 3.310214679152705
INFO:VisitTimeSeries:Epoch: 2, Loss: 3.7372892934829
INFO:VisitTimeSeries:Epoch: 3, Loss: 3.919697518646717
INFO:VisitTimeSeries:Epoch: 4, Loss: 4.501575288921595


In [22]:
len(train_indices)/100 * 3.2 / 60

19.053333333333335

In [17]:
torch.save(model, "{}/{}.pt".format(_MODEL_ROOT_PATH, model_name))

35725

In [ ]:
model = model.eval()
count = 0
total_loss = 0

for idx in test_indices:
    curr_series = test_series_df.xs(idx, level=[0,1])
    output = test_labels_df.xs(idx, level=[0,1])
    
    logit = model(utils.to_tensor(curr_series))
    total_loss += loss_fn(logit, utils.to_tensor(output))
total_loss = total_loss.data.numpy()

_LOGGER.info("Final Validation Loss: {}".format(total_loss/len(test_indices)))

In [ ]:
experiment_utils.setup_model_logger(_LOGGER, model_name, _MODEL_LOG_ROOT_PATH)

test_losses = experiment_utils.evaluate_validation_loss_template(model, loss_fn, test_series, test_labels)

_LOGGER.info("Initial Validation Loss: {}".format(np.mean(test_losses)))


training_losses = experiment_utils.train_model_template(model, loss_fn, optimizer, args.n_epoch, train_series, train_labels, _LOGGER)
torch.save(model, "{}/{}.pt".format(_MODEL_ROOT_PATH, model_name))


test_losses = experiment_utils.evaluate_validation_loss_template(model, 
                                          loss_fn, 
                                          test_series,
                                         test_labels)

_LOGGER.info("Final Validation Loss: {}".format(np.mean(test_losses)))